In [1]:
from bulk_lambert.bulk import propagate as propagate_cpu
from bulk_lambert.cuda.bulk import propagate as propagate_gpu

from astropy import time
from astropy import units as u

import numpy as np

from poliastro.ephem import Ephem
from poliastro.twobody import Orbit
from poliastro.bodies import Earth, Mars, Sun
from poliastro.util import time_range

In [2]:
date_launch = time.Time("2011-11-26 15:02", scale="utc").tdb
date_arrival = time.Time("2012-08-06 05:17", scale="utc").tdb

earth = Ephem.from_body(Earth, time_range(date_launch, end=date_arrival))
mars = Ephem.from_body(Mars, time_range(date_launch, end=date_arrival))

In [3]:
dates = time_range(start = time.Time("2016-01-01 00:00"), spacing = 7 * u.d, periods = 52 * 10)

In [4]:
ss_earth = Orbit.from_ephem(Sun, earth, date_launch)
ss_mars = Orbit.from_ephem(Sun, mars, date_arrival)

earth_rr, earth_vv = propagate_cpu(ss_earth, dates)
mars_rr, mars_vv = propagate_cpu(ss_mars, dates)

In [5]:
ss_earth = Orbit.from_ephem(Sun, earth, date_launch)
ss_mars = Orbit.from_ephem(Sun, mars, date_arrival)

earth_rrG, earth_vvG = propagate_gpu(ss_earth, dates)
mars_rrG, mars_vvG = propagate_gpu(ss_mars, dates)

In [9]:
%timeit propagate_cpu(ss_earth, dates)

2.28 ms ± 30.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%timeit propagate_gpu(ss_earth, dates)

1.67 ms ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
all([
    np.all(np.isclose(earth_rr.value, earth_rrG.value))
    for a, b in (
        (earth_rr.value, earth_rrG.value),
        (mars_rr.value, mars_rrG.value),
        (earth_vv.value, earth_vvG.value),
        (mars_vv.value, mars_vvG.value),
    )
])

True